# Transform Orders Data - Explode Array
1. Access elements from the JSON object
2. Deduplicate array elements
3. Explode arrays
4. Write the transformed data to Silver schema

In [0]:
SELECT *
FROM udemyttstorage_catalog.silver.orders_json;

1. Access elements from the JSON object

In [0]:
SELECT 
    json_value.order_id,
    json_value.order_status,
    json_value.order_status,
    json_value.payment_method,
    json_value.total_amount,
    json_value.transaction_timestamp,
    json_value.customer_id,
    json_value.items
FROM udemyttstorage_catalog.silver.orders_json;

2. Deduplicate array elements\
  [Documentation for array_distinct](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/functions/array_distinct)

In [0]:
%sql
SELECT 
    json_value.order_id,
    json_value.order_status,
    json_value.order_status,
    json_value.payment_method,
    json_value.total_amount,
    json_value.transaction_timestamp,
    json_value.customer_id,
    array_distinct(json_value.items) AS items
FROM udemyttstorage_catalog.silver.orders_json;

3. Explode arrays \
  [Documentation for explode](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/functions/explode)

In [0]:
%sql
SELECT 
    json_value.order_id,
    json_value.order_status,
    json_value.order_status,
    json_value.payment_method,
    json_value.total_amount,
    json_value.transaction_timestamp,
    json_value.customer_id,
    explode(array_distinct(json_value.items)) AS item
FROM silver.orders_json;

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW orders_temp_view
AS
SELECT 
    json_value.order_id,
    json_value.order_status,
    json_value.payment_method,
    json_value.total_amount,
    json_value.transaction_timestamp,
    json_value.customer_id,
    explode(array_distinct(json_value.items)) AS item
FROM silver.orders_json;

In [0]:
%sql
SELECT
  order_id,
  order_status
  payment_method,
  total_amount,
  transaction_timestamp,
  customer_id,
  item.category,
  item.details.brand,
  item.details.color,
  item.item_id,
  item.name,
  item.price,
  item.quantity
FROM orders_temp_view;

4. Write the transformed data to Silver schema

In [0]:
%sql
CREATE OR REPLACE TABLE udemyttstorage_catalog.silver.orders
AS
SELECT
  order_id,
  order_status
  payment_method,
  total_amount,
  transaction_timestamp,
  customer_id,
  item.category,
  item.details.brand,
  item.details.color,
  item.item_id,
  item.name,
  item.price,
  item.quantity
FROM orders_temp_view;